In [1]:
import numpy as np
import pandas as pd
import joblib
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [2]:
path = '../../data/job_data.csv'
jobs = pd.read_csv(path)
jobs.head(3)

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72


In [3]:
len(jobs)

22000

In [4]:
#drop some rows without skills information
jobs = jobs.dropna(subset=['skills','jobtitle'])
jobs = jobs.drop(jobs[jobs['skills']=='SEE BELOW'].index)
jobs = jobs.reset_index()

jobs['skills'] = jobs['skills'].str.lower()
    
jobs = jobs[~jobs['skills'].str.contains('description')]
jobs = jobs[~jobs['skills'].str.contains('the')]
jobs = jobs[~jobs['skills'].str.contains('you')]

jobs['jobtitle'] = jobs['jobtitle'].str.lower()
jobs = jobs.reset_index()
jobs.head(3)

,level_0,index,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,0,1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",information security engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
1,1,2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",business solutions architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"enterprise solutions architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
2,3,4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",devops engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"configuration management, developer, linux, ma...",45efa1f6bc65acc32bbbb953a1ed13b7


In [8]:
jobs[['skills','jobtitle']]

,skills,jobtitle
0,"linux/unix, network monitoring, incident respo...",information security engineer
1,"enterprise solutions architecture, business in...",business solutions architect
2,"configuration management, developer, linux, ma...",devops engineer
3,"fico, ar, ap, asset management, haha",sap fico architect
4,"cisco, dns, http, networking, network engineer...",network engineer
...,...,...
21095,"ui/ux mobile apps, interaction design, digital...",web designer
21096,"javascript, html5, css3, bootstrap, ajax, reac...",senior front end web developer - full time at ...
21097,"sdlc, alm, sql, t-sql, redgate, team foundatio...",qa analyst
21098,"python, ruby, go, clojure, java, nosql-databas...",tech lead-full stack


In [34]:
joblib.dump(jobs,'../model_save/cleaned_jobs_weight.pkl')

['../model_save/cleaned_jobs_weight.pkl']

In [30]:
ps = PorterStemmer()
all_stopwords = stopwords.words('english')

In [19]:
for i in range(len(jobs)):
    temp = re.sub('[^a-zA-Z]', ' ', jobs["jobtitle"][i])
    temp = temp.split()
    temp = [ps.stem(word) for word in temp if not word in set(all_stopwords)]
    temp = ' '.join(temp)
    jobs["jobtitle"][i] = temp

<ipython-input-19-5e5d6d338c78>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs["jobtitle"][i] = temp


In [23]:
for i in range(len(jobs)):
    temp = re.sub('[^a-zA-Z]', ' ', jobs["skills"][i])
    temp = temp.split()
    temp = [ps.stem(word) for word in temp if not word in set(all_stopwords)]
    temp = ' '.join(temp)
    jobs["skills"][i] = temp

<ipython-input-23-26bc5a94489c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs["skills"][i] = temp


In [24]:
jobs.head(3)

,level_0,index,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,0,1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",inform secur engin,1 week ago,Telecommuting not available|Travel not required,NaN,linux unix network monitor incid respons syste...,8aec88cba08d53da65ab99cf20f6f9d9
1,1,2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",busi solut architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,enterpris solut architectur busi intelig repor...,46baa1f69ac07779274bcd90b85d9a72
2,3,4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",devop engin,48 minutes ago,Telecommuting not available|Travel not required,NaN,configur manag develop linux manag process eng...,45efa1f6bc65acc32bbbb953a1ed13b7


In [25]:
joblib.dump(jobs,'../model_save/cleaned_jobs.pkl')

['../model_save/cleaned_jobs.pkl']